In [1]:
# Add locations of modules
push!(LOAD_PATH, "../src/")

3-element Array{Any,1}:
 "/Applications/Julia-0.6.app/Contents/Resources/julia/local/share/julia/site/v0.6"
 "/Applications/Julia-0.6.app/Contents/Resources/julia/share/julia/site/v0.6"      
 "../src/"                                                                         

In [2]:
using GraphicalModelLearning
using LightGraphs
using Cairo
using Junet

include("beta_scaling.jl")

INFO: Recompiling stale cache file /Users/gogol/.julia/lib/v0.6/GraphicalModelLearning.ji for module GraphicalModelLearning.

Use "FactorGraph{T}(...) where T" instead.


get_M_opt_glauber_dynamics (generic function with 5 methods)

### Experiments for Spin History Only case

#### Create the dictionary over samples

In [3]:
# Define Graphical Model
model = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])

n_samples = 100000

# Gibbs Samples
samples_T, samples_mixed = gibbs_sampling(model, n_samples, T_regime())
print("done")

INFO: assuming spin alphabet
INFO: using Glauber dynamics v1 to generate T-regime samples


done

In [5]:
samples_T

48×8 Array{Int64,2}:
  962  1   1  -1   1  -1  -1   1
 1038  2   1   1  -1   1  -1  -1
  616  3   1  -1   1   1  -1   1
  973  2  -1  -1   1  -1   1   1
 1001  1   1   1   1  -1   1   1
  164  1   1  -1   1   1  -1   1
 3383  3  -1  -1   1  -1  -1  -1
  996  1  -1  -1   1   1  -1   1
 7688  1  -1  -1  -1  -1  -1  -1
 3415  3   1   1  -1   1   1   1
  583  3  -1   1   1  -1   1   1
  939  1   1   1  -1  -1   1  -1
 7374  1   1   1   1   1   1   1
    ⋮                  ⋮        
 2359  3   1   1  -1   1   1  -1
  128  1  -1   1   1  -1   1   1
  951  1  -1   1  -1   1   1  -1
 2229  3  -1  -1   1  -1  -1   1
  590  3  -1   1  -1  -1   1   1
 1004  1  -1  -1  -1   1  -1  -1
  206  1  -1   1  -1  -1   1  -1
  994  1  -1   1   1   1   1   1
  953  2  -1   1   1  -1  -1   1
 1019  2   1  -1  -1   1   1  -1
  580  3   1  -1  -1   1  -1   1
  145  1   1  -1  -1   1  -1  -1

In [6]:
samples_T2 = copy(samples_T)
samples_T2[48,:] = []

LoadError: [91mDimensionMismatch("tried to assign 0-element array to 1×8 destination")[39m

In [8]:
# Create a dictionary over samples_T
dict_samples_sho = Dict{UInt8, Array{Int64, 2}}()
num_spins = 3

# Node number is the key of the dictionary
# Should include those which are flips and those which are not
# For the cases where there are flips, we know node i is being updated because this is single-site dynamics and we see a change
# Where there are no flips, we don't know which node is being updated so include all possibilities 

# For each dict[key], array should be of form (n_samples, FLAG_flip, \sigma^{(t)}, \sigma^{(t+1)})

# First get all those samples where there is no flip
ind_samples_T_no_flip = []
ind_samples_T_flip = []

for ind_config = 1:size(samples_T,1)
    if dot(samples_T[ind_config,3:(2+num_spins)],samples_T[ind_config,(3+num_spins):end]) == num_spins
        push!(ind_samples_T_no_flip,ind_config)
    else
        push!(ind_samples_T_flip,ind_config)
    end
end

samples_T_flip = samples_T[ind_samples_T_flip,:]
samples_T_no_flip = samples_T[ind_samples_T_no_flip,:]

24×8 Array{Int64,2}:
  616  3   1  -1   1   1  -1   1
  164  1   1  -1   1   1  -1   1
 7688  1  -1  -1  -1  -1  -1  -1
  583  3  -1   1   1  -1   1   1
 7374  1   1   1   1   1   1   1
 4890  2  -1  -1   1  -1  -1   1
  154  2   1  -1   1   1  -1   1
  115  2  -1   1  -1  -1   1  -1
 5176  3  -1  -1  -1  -1  -1  -1
 7503  2   1   1   1   1   1   1
 7756  2  -1  -1  -1  -1  -1  -1
  599  3   1  -1  -1   1  -1  -1
 4791  1  -1  -1   1  -1  -1   1
 5044  3   1   1   1   1   1   1
 4801  2   1   1  -1   1   1  -1
  186  2   1  -1  -1   1  -1  -1
  537  3  -1   1  -1  -1   1  -1
  199  2  -1   1   1  -1   1   1
 4722  1   1   1  -1   1   1  -1
 2359  3   1   1  -1   1   1  -1
  128  1  -1   1   1  -1   1   1
 2229  3  -1  -1   1  -1  -1   1
  206  1  -1   1  -1  -1   1  -1
  145  1   1  -1  -1   1  -1  -1

In [9]:
display(samples_T_flip)

24×8 Array{Int64,2}:
  962  1   1  -1   1  -1  -1   1
 1038  2   1   1  -1   1  -1  -1
  973  2  -1  -1   1  -1   1   1
 1001  1   1   1   1  -1   1   1
 3383  3  -1  -1   1  -1  -1  -1
  996  1  -1  -1   1   1  -1   1
 3415  3   1   1  -1   1   1   1
  939  1   1   1  -1  -1   1  -1
 1044  1   1  -1  -1  -1  -1  -1
 3437  3  -1  -1  -1  -1  -1   1
 1046  2  -1   1  -1  -1  -1  -1
 1012  2   1   1   1   1  -1   1
  601  3   1  -1   1   1  -1  -1
 1026  2   1  -1   1   1   1   1
  617  3  -1   1   1  -1   1  -1
 1031  2  -1  -1  -1  -1   1  -1
 3422  3   1   1   1   1   1  -1
  951  1  -1   1  -1   1   1  -1
  590  3  -1   1  -1  -1   1   1
 1004  1  -1  -1  -1   1  -1  -1
  994  1  -1   1   1   1   1   1
  953  2  -1   1   1  -1  -1   1
 1019  2   1  -1  -1   1   1  -1
  580  3   1  -1  -1   1  -1   1

In [10]:
display(samples_T_no_flip)

24×8 Array{Int64,2}:
  616  3   1  -1   1   1  -1   1
  164  1   1  -1   1   1  -1   1
 7688  1  -1  -1  -1  -1  -1  -1
  583  3  -1   1   1  -1   1   1
 7374  1   1   1   1   1   1   1
 4890  2  -1  -1   1  -1  -1   1
  154  2   1  -1   1   1  -1   1
  115  2  -1   1  -1  -1   1  -1
 5176  3  -1  -1  -1  -1  -1  -1
 7503  2   1   1   1   1   1   1
 7756  2  -1  -1  -1  -1  -1  -1
  599  3   1  -1  -1   1  -1  -1
 4791  1  -1  -1   1  -1  -1   1
 5044  3   1   1   1   1   1   1
 4801  2   1   1  -1   1   1  -1
  186  2   1  -1  -1   1  -1  -1
  537  3  -1   1  -1  -1   1  -1
  199  2  -1   1   1  -1   1   1
 4722  1   1   1  -1   1   1  -1
 2359  3   1   1  -1   1   1  -1
  128  1  -1   1   1  -1   1   1
 2229  3  -1  -1   1  -1  -1   1
  206  1  -1   1  -1  -1   1  -1
  145  1   1  -1  -1   1  -1  -1

In [13]:
samples_T_no_flip[:,2] = 0
# Now create the dictionary
for current_spin = 1:num_spins
    samples_current_spin = samples_T_flip[find(isequal(current_spin),samples_T_flip[:,2]),:]
    samples_current_spin[:,2] = 1
    dict_samples_sho[current_spin] = vcat(samples_current_spin,samples_T_no_flip)
end
print("done")

done

In [14]:
dict_samples_sho

Dict{UInt8,Array{Int64,2}} with 3 entries:
  0x02 => [1038 1 … -1 -1; 973 1 … 1 1; … ; 206 0 … 1 -1; 145 0 … -1 -1]
  0x03 => [3383 1 … -1 -1; 3415 1 … 1 1; … ; 206 0 … 1 -1; 145 0 … -1 -1]
  0x01 => [962 1 … -1 1; 1001 1 … 1 1; … ; 206 0 … 1 -1; 145 0 … -1 -1]

In [15]:
dict_samples_sho[1]

32×8 Array{Int64,2}:
  962  1   1  -1   1  -1  -1   1
 1001  1   1   1   1  -1   1   1
  996  1  -1  -1   1   1  -1   1
  939  1   1   1  -1  -1   1  -1
 1044  1   1  -1  -1  -1  -1  -1
  951  1  -1   1  -1   1   1  -1
 1004  1  -1  -1  -1   1  -1  -1
  994  1  -1   1   1   1   1   1
  616  0   1  -1   1   1  -1   1
  164  0   1  -1   1   1  -1   1
 7688  0  -1  -1  -1  -1  -1  -1
  583  0  -1   1   1  -1   1   1
 7374  0   1   1   1   1   1   1
    ⋮                  ⋮        
 4791  0  -1  -1   1  -1  -1   1
 5044  0   1   1   1   1   1   1
 4801  0   1   1  -1   1   1  -1
  186  0   1  -1  -1   1  -1  -1
  537  0  -1   1  -1  -1   1  -1
  199  0  -1   1   1  -1   1   1
 4722  0   1   1  -1   1   1  -1
 2359  0   1   1  -1   1   1  -1
  128  0  -1   1   1  -1   1   1
 2229  0  -1  -1   1  -1  -1   1
  206  0  -1   1  -1  -1   1  -1
  145  0   1  -1  -1   1  -1  -1

#### "Try" to learn the structure

In [11]:
# M-Regime Sampling
n_samples = 20000
samples_T, samples_mixed = gibbs_sampling(model, n_samples, M_regime())

# Create a dictionary over samples_T
dict_samples_sho = Dict{Int, Array{Int64, 2}}()
num_spins = 3

# Node number is the key of the dictionary
# Should include those which are flips and those which are not
# For the cases where there are flips, we know node i is being updated because this is single-site dynamics and we see a change
# Where there are no flips, we don't know which node is being updated so include all possibilities 

# For each dict[key], array should be of form (n_samples, FLAG_flip, \sigma^{(t)}, \sigma^{(t+1)})

# First get all those samples where there is no flip
ind_samples_T_no_flip = []
ind_samples_T_flip = []

for ind_config = 1:size(samples_T,1)
    if dot(samples_T[ind_config,3:(2+num_spins)],samples_T[ind_config,(3+num_spins):end]) == num_spins
        push!(ind_samples_T_no_flip,ind_config)
    else
        push!(ind_samples_T_flip,ind_config)
    end
end

samples_T_flip = samples_T[ind_samples_T_flip,:]
samples_T_no_flip = samples_T[ind_samples_T_no_flip,:]

samples_T_no_flip[:,2] = 0
# Now create the dictionary
for current_spin = 1:num_spins
    samples_current_spin = samples_T_flip[find(isequal(current_spin),samples_T_flip[:,2]),:]
    samples_current_spin[:,2] = 1
    dict_samples_sho[current_spin] = vcat(samples_current_spin,samples_T_no_flip)
end
print("done")

INFO: using Glauber dynamics v1 to generate M-regime samples


done

In [13]:
learned_gm = learn_glauber_dynamics_sho(dict_samples_sho, n_samples)

INFO: using JuMP for RISE to learn Glauber dynamics from SHO samples


3×3 Array{Float64,2}:
 -0.0467434  0.902008   0.054602 
  0.902008   0.0375095  0.102806 
  0.054602   0.102806   0.0151503

In [9]:
true_gm = [0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0]

3×3 Array{Float64,2}:
 0.0  0.9  0.1
 0.9  0.0  0.1
 0.1  0.1  0.0

In [12]:
learned_gm = learn_glauber_dynamics(samples_T)

INFO: using JuMP for RISE to learn Glauber dynamics


3×3 Array{Float64,2}:
 -0.0255645  0.891516    0.0793898 
  0.891516   0.0101021   0.0987405 
  0.0793898  0.0987405  -0.00179326

#### More complicated case